In [ ]:
import onnxruntime               # 导入onnxruntime库，它是运行ONNX模型的引擎，可以高效地执行深度学习模型的推理。
import numpy as np               # 导入numpy库，用于进行数值计算，尤其擅长处理数组和矩阵。
from PIL import Image            # 从PIL (Pillow) 库导入Image模块，用于图像处理，如打开、调整大小和格式转换。


# 加载ONNX模型  2分
ort_session = onnxruntime.InferenceSession('mnist.onnx')
# 使用`onnxruntime.InferenceSession`函数加载名为 'mnist.onnx' 的预训练ONNX模型。
# 这个模型通常是用于手写数字识别的MNIST数据集上训练的。
# `ort_session` 对象是与加载模型交互的接口，允许我们执行模型的推理。

# 加载图像 2分
image = Image.open('img_test.png').convert('L')  # 转为灰度图
# 使用PIL的`Image.open()`函数打开名为 'img_test.png' 的图片文件。
# `.convert('L')` 是一个关键步骤，它将图片转换为灰度图。
# 'L' 模式代表亮度（Luminance），即单通道的灰度图。MNIST模型通常期望28x28的灰度图片作为输入。

#图像预处理
image = image.resize((28, 28))  # 调整大小为MNIST模型的输入尺寸2分
# 将加载并转换为灰度的图片调整大小为28x28像素，这是MNIST手写数字识别模型标准输入尺寸。
# `resize()` 方法会根据指定的宽度和高度调整图片大小。

image_array = np.array(image, dtype=np.float32)  # 转为numpy数组2分
# 将PIL Image对象转换为NumPy数组。
# `np.array(image)` 得到一个形状为 (28, 28) 的NumPy数组，其元素代表像素值。
# `dtype=np.float32` 将数组元素的数据类型指定为32位浮点数，这是深度学习模型通常要求的输入类型。

image_array = np.expand_dims(image_array, axis=0)  # 添加batch维度2分
# `np.expand_dims()` 函数用于在指定轴（维度）上插入一个新轴。
# 这里 `axis=0` 表示在数组的最前面添加一个新维度。
# 原始 `image_array` 形状是 (28, 28)，添加批次维度后变为 (1, 28, 28)。
# `1` 表示批次大小为1，因为模型通常期望输入是一个批次（Batch）的图像，即使我们只处理一张图片。

image_array = np.expand_dims(image_array, axis=0)  # 添加通道维度2分
# 再次使用 `np.expand_dims()` 在数组的第二个位置（即批次维度之后）添加一个新维度。
# 上一步形状是 (1, 28, 28)，添加通道维度后变为 (1, 1, 28, 28)。
# `1` 在这里表示图像有一个颜色通道，即它是灰度图。
# 多数深度学习模型（特别是卷积神经网络）期望输入形状是 `(batch_size, channels, height, width)`。

#返回模型输入列表 2分
ort_inputs = {ort_session.get_inputs()[0].name: image_array}
# 创建一个字典 `ort_inputs`，用于作为ONNX模型的输入。
# 字典的键是模型第一个输入节点的名称，通过 `ort_session.get_inputs()[0].name` 获取。
# 字典的值是经过所有预处理步骤的图像NumPy数组 `image_array`。

# 执行预测 2分
ort_outs = ort_session.run(None, ort_inputs)
# 使用 `ort_session.run()` 方法执行模型的推理（prediction）。
# 第一个参数 `None` 表示我们希望获取模型所有的输出。
# 第二个参数 `ort_inputs` 是之前准备好的输入字典。
# `ort_outs` 将是一个列表，包含模型的所有输出结果。对于分类模型，这通常是各个类别的logits（或概率）。

# 获取预测结果 2分
predicted_class = np.argmax(ort_outs[0][0])
# `ort_outs[0]` 获取模型输出列表中的第一个元素，这通常是包含所有类别的得分或概率的数组。
# 对于单个图像的分类任务，这个数组的形状可能为 `(1, num_classes)`，其中 `1` 是批次大小。
# `ort_outs[0][0]` 因此获取了第一个（也是唯一一个）样本的输出分数。
# `np.argmax()` 函数返回沿给定轴的最大值的索引。在这里，它会找到得分最高的那个类别的索引。
# 这个索引 `predicted_class` 就是模型识别出的数字。

# 输出预测结果
print(f"Predicted class: {predicted_class}")
# 打印最终的预测结果，即模型认为图片中显示的数字是哪一个。
